In [ ]:
# # Prepare Google Colab Environement and build handmade library
# !git clone https://github.com/kaenova/Headline_Detection.git
# %cd "/content/Headline_Detection"

# !make lib

# %cd "/content/"

# print("Please upload '4. Processed_2023_04_21.zip'")
# from google.colab import files
# files.upload()

# !unzip "/content/4. Processed_2023_04_21.zip"

# print("Please upload .env")
# from google.colab import files
# files.upload()

In [1]:
# # Reset Google Colab Environment
# %cd ..
# !rm -fr Headline_Detection

# Load Config and Env

In [1]:
# Load env for S3 uploads
from dotenv import load_dotenv
from kaelib.s3 import check_required_env_vars
import os
load_dotenv()
check_required_env_vars()

# Load Dataset

In [3]:
import os
import pandas as pd

environment = 'colab'
if environment == 'local':
    data_path = "../../data/4. Processed/"
elif environment == 'colab':
    data_path = "/content/4. Processed/"
else:
    raise ImportError("Data environment")


folders = os.listdir(data_path)
scenario_data = {}

for folder in folders:
    current_data_path = f"{data_path}/{folder}"
    if not os.path.isdir(current_data_path):
        continue
    files = os.listdir(current_data_path)
    data_dict = {}
    for file in files:
        current_file = f"{current_data_path}/{file}"
        data_dict[file] = pd.read_csv(current_file)
    scenario_data[int(folder)] = data_dict

In [4]:
import torch
import pandas
import typing
from torch.utils.data import Dataset
from kaelib.processor import TextProcessingPipeline


class TextClassificationDataset(Dataset):
    def __init__(
        self,
        df: "pandas.DataFrame",
        x_column_name: "str" = "tweet",
        y_column_name: "str" = "labels",
        preprocessor: "typing.Optional[TextProcessingPipeline]" = None,
    ):
        self.x = df[x_column_name].astype(str).to_list()
        self.y = torch.tensor(df[y_column_name].astype(int).to_list())
        assert len(self.x) == len(self.y)
        self.preprocessor = preprocessor

    def __len__(self):
        return len(self.x)

    def _process_idx_text(self, idx):
        data = self.x[idx]
        if type(idx) is not slice:
            data = [self.x[idx]]
        if self.preprocessor is not None:
            data = self.preprocessor.process_corpus(data)
        return data

    def __getitem__(self, idx):
        processed_corpus = self._process_idx_text(idx)
        return processed_corpus, self.y[idx]

    def __repr__(self) -> str:
        return "\n".join([f"{self.x[i]} : {self.y[i]}" for i in range(5)])


scenario_datasets = {}
for scenario in scenario_data:
    data = scenario_data[scenario]
    datasets = {}
    for data_type in data:
        datasets[data_type] = TextClassificationDataset(data[data_type])
    scenario_datasets[scenario] = datasets

# Modelling

In [5]:
# Hyperparameters
"""
https://arxiv.org/pdf/1810.04805.pdf

We use a batch size of 32 and fine-tune for 3
epochs over the data for all GLUE tasks. For each
task, we selected the best fine-tuning learning rate
(among 5e-5, 4e-5, 3e-5, and 2e-5)
"""
hyper_params = {
    'model_name': "indolem/indobertweet-base-uncased",
    'seq_length': 256,
    'out_feature': 2,
    'learning_rate': 2e-5,
    'batch_size': 16
}

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning.pytorch as pl
from transformers import BertForSequenceClassification, BertTokenizerFast
from torchmetrics.classification import F1Score, Accuracy, Recall, Precision


class BERTClassifier(pl.LightningModule):
    def __init__(
        self,
        huggingface_model_name: "str" = "indolem/indobertweet-base-uncased",
        seq_length: "int" = 256,
        out_feature: "int" = 2,
        pad_sequence: "bool" = True,
    ):
        super().__init__()
        self.seq_length = seq_length
        self.pad_sequence = pad_sequence
        self.tokenizer = BertTokenizerFast.from_pretrained(huggingface_model_name)
        self.huggingface_model = BertForSequenceClassification.from_pretrained(
            huggingface_model_name,
            num_labels=out_feature,
            problem_type="multi_label_classification",
        ).to(self.device)

        self.f1_scorer = F1Score(task="multiclass", num_classes=out_feature)
        self.accuracy_scorer = Accuracy(task="multiclass", num_classes=out_feature)
        self.precision_scorer = Precision(task="multiclass", num_classes=out_feature)
        self.recall_scorer = Recall(task="multiclass", num_classes=out_feature)

    def _forward_huggingface_tokenizers(self, x: "list[str]"):
        for sentence in x:
            sentence_seq = sentence.split(" ")
            if len(sentence_seq) > self.seq_length:
                sentence_seq = sentence_seq[: self.seq_length]
            if self.pad_sequence:
                while len(sentence_seq) < self.seq_length:
                    sentence_seq.append("[PAD]")
        tokens = self.tokenizer(
            x,
            max_length=512, # Max BERT tokens
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = tokens["input_ids"].to(self.device)  # type: ignore
        attention_mask = tokens["attention_mask"].to(self.device)  # type: ignore
        return input_ids, attention_mask

    def forward(self, x: "list[str]") -> "torch.Tensor":
        # Prepare str
        input_ids, attention_mask = self._forward_huggingface_tokenizers(x)
        logits = self.huggingface_model(input_ids=input_ids, attention_mask=attention_mask).logits  # type: ignore
        return logits

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("training_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("training_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("training_accuracy", accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("validation_loss", loss, prog_bar=True)
        f1_score = self.f1_scorer(pred, y)
        self.log("validation_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("validation_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("validation_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("validation_recall", recall, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("test_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("test_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("test_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("test_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("test_recall", recall, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=hyper_params['learning_rate'])
        return optimizer


In [7]:
model_module = BERTClassifier
def create_model():
    return model_module(hyper_params['model_name'], hyper_params['seq_length'], hyper_params['out_feature'])

def load_model(path):
    return model_module.load_from_checkpoint(path, huggingface_model_name=hyper_params['model_name'], seq_length=hyper_params['seq_length'], out_feture=hyper_params['out_feature'])

In [8]:
model = create_model()
model(['test nama kamu siapaa?','halo']).shape

Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

torch.Size([2, 2])

# Train, Val, Test

In [3]:
available_cpus = 0
model_name = "indobert"
tuning = False

In [14]:
import datetime
import random
import torch
import numpy as np

from torch.utils.data import DataLoader
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

now = datetime.datetime.now()
timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')

for scenario in scenario_datasets:
    np.random.seed(2023)
    random.seed(2023)
    torch.manual_seed(2023)

    run_id = f"{timestamp}_scenario_{scenario}"

    if scenario in [0]:
        continue

    if tuning:
        if scenario in [2,3,4,5,6]:
            continue
        run_id += "_tuning"

    model = create_model()

    # Dataset
    train_dataset = scenario_datasets[scenario]["train.csv"]
    test_dataset = scenario_datasets[scenario]["test.csv"]
    # validation_dataset = scenario_datasets[scenario]["validation.csv"]
    train_loader = DataLoader(train_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)
    # validation_loader = DataLoader(validation_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)
    test_loader = DataLoader(test_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)

    tb_log = TensorBoardLogger("tensorboard", model_name, run_id)

    checkpoint_callback_val = ModelCheckpoint(monitor="validation_loss", filename='val_{epoch}-{validation_loss:.4f}')
    checkpoint_callback_train = ModelCheckpoint(monitor="training_loss", filename='train_{epoch}-{training_loss:.4f}')
    early_stop_callback = EarlyStopping(
        monitor="validation_loss", min_delta=1e-3, patience=5, mode="min"
    )

    trainer = pl.Trainer(
        logger=tb_log,
        log_every_n_steps=2, 
        callbacks=[early_stop_callback, checkpoint_callback_val, checkpoint_callback_train]
    )
    trainer.fit(
        model=model, train_dataloaders=train_loader, val_dataloaders=test_loader
    )
    model = load_model(checkpoint_callback_val.best_model_path)
    test_acc = trainer.test(model=model, dataloaders=test_loader)[0]['test_accuracy']
    tb_log.log_hyperparams(hyper_params, {'hp_metric': test_acc})


Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/ind

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9895012974739075     │
│          test_f1          │    0.9895012974739075     │
│         test_loss         │    0.05422711744904518    │
│      test_precision       │    0.9895012974739075     │
│        test_recall        │    0.9895012974739075     │
└───────────────────────────┴───────────────────────────┘

# Upload Logging and Checkpoint to S3

In [15]:
!find "/content/tensorboard" -type f -name 'train_epoch=*' -delete

In [4]:
import os
from kaelib.s3_threading import upload_folder_to_s3_all_threads
s3_folders = os.path.join(os.getenv("BUCKET_FOLDER"), model_name)
upload_folder_to_s3_all_threads(f"tensorboard/{model_name}", s3_folders)

Uploading:   3%|▎         | 1/35 [00:00<00:26,  1.30false/s, file=Log and Checkpoints/2023_04_21/indobert/2023-04-27_04-48-48_scenario_2/events.out.tfevents.1682574053.84a92aafab42.3957.11]

Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_2/events.out.tfevents.1682574053.84a92aafab42.3957.11 as it already exists in S3


Uploading:  31%|███▏      | 11/35 [00:01<00:01, 12.32false/s, file=Log and Checkpoints/2023_04_21/indobert/2023-04-27_04-48-48_scenario_5/events.out.tfevents.1682574991.84a92aafab42.3957.13]

Skipping upload of tensorboard/indobert/2023-04-27_04-26-31_scenario_1_tuning/events.out.tfevents.1682570845.84a92aafab42.3957.1 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_4/events.out.tfevents.1682571867.84a92aafab42.3957.6 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_5/events.out.tfevents.1682574054.84a92aafab42.3957.12 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_1/events.out.tfevents.1682577324.84a92aafab42.3957.20 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_6/events.out.tfevents.1682576074.84a92aafab42.3957.16 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-26-31_scenario_1_tuning/hparams.yaml as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_1/events.out.tfevents.1682577323.84a92aafab42.3957.19 as i

Uploading:  43%|████▎     | 15/35 [00:01<00:01, 14.61false/s, file=Log and Checkpoints/2023_04_21/indobert/2023-04-27_04-48-48_scenario_3/hparams.yaml]                                      WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: is3.cloudhost.id. Connection pool size: 10


...

Uploading:  71%|███████▏  | 25/35 [00:02<00:01,  8.93false/s, file=Log and Checkpoints/2023_04_21/indobert/2023-04-27_04-48-48_scenario_2/hparams.yaml]

Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_2/hparams.yaml as it already exists in S3


Uploading:  77%|███████▋  | 27/35 [00:03<00:01,  5.97false/s, file=Log and Checkpoints/2023_04_21/indobert/2023-04-27_04-48-48_scenario_2/events.out.tfevents.1682572963.84a92aafab42.3957.9]

Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_4/events.out.tfevents.1682572961.84a92aafab42.3957.8 as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_5/hparams.yaml as it already exists in S3
Skipping upload of tensorboard/indobert/2023-04-27_04-48-48_scenario_2/events.out.tfevents.1682572963.84a92aafab42.3957.9 as it already exists in S3


KeyboardInterrupt: ignored

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp -r "/content/tensorboard" "/content/drive/Shareddrives/_PercobaanKaenova/Tugas Akhir"